# Exercise 6. ICT Project: Communication Services and Security
### Cèsar Fernàndez Camón

#### Authors:
- Albert Pérez Datsira
- Jeongyun Lee


## Problem 2
Let’s assume a CCMP (Cipher Block Chaining Message) with 8 bits length block. The counter blocks are computed
as follows:
- ctr(0) = key >> 2
- ctr(i) = ctr(i - 1) >> 2 ⊕ 0x53, i > 0

where (>>) is a circular shift to the right and (⊕) is a XOR operation. Consider:
* key = 0x22
* information = 0x0001020304
* AES as a XOR operation

Probe that the ciphered information is **0xAA522FB151** and its corresponding MIC is **0x26**.
Check that the same schema also works as a decipher.

In [3]:
blockLength = 8
x = 0x53

key = 0x22
information = 0x0001020304
informationArray = [0x00,0x01,0x02,0x03,0x04]

cipheredInformation = 0xAA522FB151
MIC = 0x26

shiftBy = 1
IV = int('00000000', 2)

In [4]:
print("\033[1m{0}\033[0m {1}"    .format("          Key:  ", key))
print("\033[1m{0}\033[0m {1}"    .format("         Info:  ", information))
print("\033[1m{0}\033[0m {1}"    .format("   Info array:  ", informationArray))
print("\033[1m{0}\033[0m {1}"    .format("          MIC:  ", MIC))
print("\033[1m{0}\033[0m {1}"    .format("Ciphered info:  ", cipheredInformation))
print("\033[1m{0}\033[0m {1:08b}".format("           IV:  ", IV))
print("\033[1m{0}\033[0m {1}"    .format("            x:  ", x))
print("\033[1m{0}\033[0m {1}"    .format(" Block length:  ", blockLength))
print("\033[1m{0}\033[0m {1}"    .format("     Shift by:  ", shiftBy))

          Key:   34
         Info:   16909060
   Info array:   [0, 1, 2, 3, 4]
          MIC:   38
Ciphered info:   731523297617
           IV:   00000000
            x:   83
 Block length:   8
     Shift by:   1


## Utils
### Circular shift right
This function performs a bitwise rotation assuming that the bits are rotated as if the left and right ends of the register were joined (circular ratation).

In [5]:
def circShift(val):
    tmp = val & shiftBy
    return (val >> shiftBy) | (tmp << (blockLength - shiftBy))

### CTR (AES block cipher mode)


In [6]:
def ctr(val):
    ctr = []
    for i in range(0, 5):
        if (i == 0):
            tmp = circShift(circShift(val))
        else:
            tmp = circShift(circShift(ctr[i - 1])) ^ x # XOR operation
        ctr.append(tmp)
    return ctr

## Problem execution
### Calculation CTR

In [7]:
CTR = ctr(key)
print(CTR)
for v in CTR: print("{0:08b} ".format(v), end='')

[136, 113, 15, 144, 119]
10001000 01110001 00001111 10010000 01110111 

### Data cyphering
Applying the CTR and the AES key into a XOR operation, the result of which will reach another XOR operation with each block of the corresponding data.

In [8]:
cyphered = []

for i in range(len(CTR)):
  tmp = informationArray[i] ^ (CTR[i] ^ key)
  cyphered.append(tmp)

for v in cyphered: print("{0:08b} ".format(v), end='')

10101010 01010010 00101111 10110001 01010001 

## MIC calculation (checksum)
MIC results from the CBC taking into account both the IV and the key in the computation. But, the result for the last block should be equal to the desired MIC.

In [9]:
mic = []

firstBlock = informationArray[0] ^ IV ^ key
mic. append(firstBlock)

for i in range(1, len(informationArray)):
    tmp = informationArray[i] ^ mic[i - 1] ^ key
    mic.append(tmp)

for v in mic: print("{0:08b} ".format(v), end='')  

00100010 00000001 00100001 00000000 00100110 

## Data decyphering
Applying an XOR operation between the cyphered data, the CTR and the AES key.

In [10]:
decyphered = []

for i in range(len(CTR)):
    tmp = cyphered[i] ^(CTR[i] ^ key)
    decyphered.append(tmp)

for v in decyphered: print("{0:08b} ".format(v), end='')    

00000000 00000001 00000010 00000011 00000100 

## Result comparison
Once all the computations are done, let's focus on what results were obtained since the aim is to get as descyphered the same original information provided as input data.